In [1]:
# !pip install transformers accelerate torch sentencepiece

   ---------------------------------------- 0.0/992.0 kB ? eta -:--:--
   --------------------------------------- 992.0/992.0 kB 45.6 MB/s eta 0:00:00


In [8]:
# !pip install psycopg2 transformers accelerate torch

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 28.2 MB/s eta 0:00:00


In [70]:
# note = ul.get_clinical_note(subject_id=10000032)
# prompt = ul.sdh_prompt(note)
# response = pipe(prompt, max_new_tokens=400)[0]['generated_text']
# print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



    You are a clinical NLP assistant. Analyze the following clinical note and indicate if each of the following social determinants of health (SDOH) topics is specifically mentioned.
    
    Answer with **"Yes" or "No"** for each item, and include a **short evidence sentence**. If not mentioned, say: *There is no evidence.*

    ### Definitions:
    
    1. **Employment status**: Whether the patient is currently employed, unemployed, retired, on disability, or has a job title or income source.
    2. **Housing issues**: Any mention of homelessness, unstable housing, living in shelters, or housing concerns (e.g., can't afford rent, frequent moves).
    3. **Transportation needs**: Any reference to transportation difficulties, lack of car access, reliance on public transit, missed appointments due to transportation.
    4. **Parental status**: Whether the patient has children or dependents, or is a caregiver to minors.
    5. **Relationship status**: Whether the patient is married, div

In [1]:
import torch
import json
import pandas as pd
import utils_llm as ul
from tqdm import tqdm
from datetime import datetime
from sklearn.metrics import accuracy_score, classification_report
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, T5ForConditionalGeneration, MegatronBertForCausalLM, AutoModelForSeq2SeqLM, LlamaForCausalLM

---------------------------------------------------------

# Mistral-7B-Instruct

In [ ]:
model_id = 'I:/Mistral-7B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, pad_token_id=tokenizer.eos_token_id, device=0)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [3]:
notes = ul.get_notes_for_first_n_notes(1)
prompt = ul.sdh_single_prompt(notes[0][3])

In [4]:
output = pipe(prompt, max_new_tokens=400)[0]['generated_text']

if prompt in output:
    print(output.replace(prompt, '').strip())
else:
    print(output.strip())

- Unknown


---------------------------------------------------------

# BioMistral-7B

In [106]:
model_id = 'I:/BioMistral-7B'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50000, 
    do_sample=True,
    temperature=0.7,
    top_k=50
)

Device set to use cpu


In [107]:
notes = ul.get_notes_for_first_n_notes(1)
prompt = ul.sdh_prompt(notes[0][3])

In [ ]:
# user_input = input("You: ")

# messages = [
#     {"role": "user", "content": user_input}
# ]

# prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# response = pipe(prompt)

# generated_text = response[0]['generated_text']

# if prompt in generated_text:
#     llm_reply = generated_text.replace(prompt, "").strip()
# else:
#     llm_reply = generated_text.strip()

# print(f"LLM: {llm_reply}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


---------------------------------------------------------

# clinicalt5-large

In [24]:
model_id = 'I:/Clinical-T5-Large'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained(model_id)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
pipe = pipeline('text2text-generation', model=model, tokenizer=tokenizer, pad_token_id=tokenizer.eos_token_id, device=0)

Device set to use cuda:0


In [3]:
notes = ul.get_notes_for_first_n_notes(1)
prompt = ul.sdh_single_prompt(notes[0][3])

In [77]:
prompt = 'What drug is used in the following text: He used omeprazol.'

In [4]:
output = pipe(prompt, max_new_tokens=400)[0]['generated_text']

if prompt in output:
    print(output.replace(prompt, '').strip())
else:
    print(output.strip())

_________________________ ____ ____ _ __ _________  _____   __________ ___   _______


In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

note = "38 y/o F, single mother of 2 (4 y/o and 6 y/o), h/o HTN and anxiety presents with medication nonadherence due to unstable PT barista job (~20 h/wk) and unreliable public transport causing missed appts and work shifts. Reports 2 mo rent arrears, late-payment notice pending eviction. No personal vehicle, bus cuts limit mobility. Ex-spouse provides no support; sister OOS; one friend for occasional childcare. Limited social support. Plan: continue lisinopril 10 mg daily, add SSRI; refer to housing assistance, workforce development, bus-pass voucher, subsidized childcare, social work, and food pantry."
prompt = f"Answer step by step: \
1. Identify medication.\
2. Evaluate context.\
3. Output medications. Note: {note}"

inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
outputs = model.generate(
    **inputs,
    max_new_tokens=150,
    num_beams=4,
    no_repeat_ngram_size=2,
    early_stopping=True
)
tokenizer.decode(outputs[0], skip_special_tokens=True)


In [32]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained(model_id)

prompt = 'Q: Can methotrexate be combined with an antibiotic?'

inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)

output = model.generate(
    **inputs,
    max_new_tokens=600,
    length_penalty=1.6,
    num_beams=4,
    no_repeat_ngram_size=3,
    temperature=0.8,
    top_k=150,
    top_p=0.92,
    repetition_penalty=2.1,
    early_stopping=True
)

print(tokenizer.decode(output[0], skip_special_tokens=True))


:, Q:,it?NR an anti-inflammatory drug. Can methotrexate be combined with an antibiotic? Other(?)Homo" [DR]q is the same drug? L??? a drug?? drugs??::Q:: A Q?????Thrombocozane


In [20]:
# input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
# output_ids = model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)
# generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
# print(generated_text)

In [10]:
from transformers import AutoModel, AutoTokenizer

model_id = 'I:/modernBERT'
model = AutoModel.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)


Some weights of BertModel were not initialized from the model checkpoint at I:/modernBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
prompt = 'The patient is healthy'

In [12]:
tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)

{'input_ids': tensor([[50281,   510,  3110,   310,  5875, 50282]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [20]:
pipe = pipeline(
    task="text-classification",
    model=model_id,
    torch_dtype=torch.float16,
    device=0
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at I:/modernBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


In [22]:
pipe("The patient is healthy and happy!")

[{'label': 'LABEL_1', 'score': 0.5953161120414734}]

---------------------------------------------------------

# gatortronS (gatortronGPT -> decoder-only (GPT))

In [20]:
model_id = 'I:/gatortronS'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = MegatronBertForCausalLM.from_pretrained(model_id, is_decoder=True)#.to('cuda:0')
assert tokenizer.vocab_size <= model.config.vocab_size
# pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, pad_token_id=tokenizer.eos_token_id)

In [21]:
subject_and_hadm_ids = pd.read_csv('C:/Users/salazarda/Downloads/SDOH_MIMICIII_physio_release.csv')
subject_and_hadm_ids = list(subject_and_hadm_ids.loc[:, ['patient_id', 'note_id']].drop_duplicates().itertuples(index=False, name=None))

notes = ul.get_clinical_notes_mimic3(subject_and_hadm_ids)
notes = notes[0:5]

sdoh_output = []

for note in notes:
    meta = {
        'subject_id': note[0],
        'hadm_id': note[1],
        'row_id': note[2],
        'charttime': note[3].isoformat() if note[3] else None
    }

    outputs_per_note = meta.copy()  # Start with metadata

    for sdoh in tqdm(['Employment status', 'Housing issues', 'Transportation issues', 'Parental status', 'Relationship status', 'Social support']):
        instruction = {
            'Employment status': 'Employment status: Whether the patient is currently employed, unemployed, underemployed, disability, retired, student, or unknown. LABELS: [employed, unemployed, underemployed, disability, retired, student, unknown]',
            'Housing issues': 'Housing issues: Any mention of financial status, undomiciled, other. LABELS: [financial status, undomiciled, other, unknown]', 
            'Transportation issues': 'Transportation issues: Any reference to transportation difficulties such as distance, resources, other. LABELS: [distance, resources, other, unknown]', 
            'Parental status':'Parental status: Whether the patient has a child under 18 years old. LABELS: [yes, no, unknown]',
            'Relationship status': 'Relationship status: Whether the patient is widowed, divorced, single. LABELS: [married, partnered, widowed, divorced, single, unknown]',
            'Social support': 'Social support: It does include informal or emotional support from family members, friends, or romantic partners unless such support is clearly mediated through a formal care plan by a social worker or case manager. LABELS: [presence, absence, unknown]'
        }
        
        prompt = ul.sdh_single_prompt(note[4], sdoh, instruction[sdoh])

        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
        inputs['position_ids'] = torch.arange(0, inputs['input_ids'].size(1), dtype=torch.long).unsqueeze(0)

        
        with torch.no_grad():
            generated_ids = model.generate(
                **inputs,
                max_new_tokens=600,
                length_penalty=1.6,
                num_beams=10,
                no_repeat_ngram_size=3,
                temperature=0.8,
                do_sample=True,
                top_k=15,
                top_p=0.95,
                repetition_penalty=2.1,
                early_stopping=True
            )

        output_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        outputs_per_note[sdoh] = output_text
    
    sdoh_output.append(outputs_per_note)

    

100%|██████████| 6/6 [37:55<00:00, 379.22s/it]


In [3]:
notes = ul.get_notes_for_first_n_notes(1)
prompt = ul.sdh_prompt_guevara(notes[0][3])

In [11]:
prompt

'\n    Your are a NLP expert assistant. You will be provided with the following information:\n    1. An arbitrary text sample. The sample is delimited with triple backticks.\n    2. A categories that we want to search in the text.\n    3. The definition that help you guide to identify the specific label for that category.\n    \n    TASK (follow the steps in order):\n    1. Analyse the text and determine whether the following six Social Determinants of Health (SDOH) are explicitly stated based on the definitions below.  \n    2. For each SDOH, select exactly one label from the bracketed list.  \n       • If the note contains none of the listed concepts for that SDOH, choose “unknown”.  \n       • Do NOT invent new labels, combine labels, or add explanations.  \n    3. Return your answer only as a JSON object with six keys (one per SDOH) and values equal to the chosen label.  \n       - Do not include any additional text or commentary.\n    \n    SDOH term and its definition:\n    1. Em

In [ ]:
with torch.no_grad():
    output_ids = model.generate(**inputs, max_new_tokens=200)

IndexError: index out of range in self

In [6]:
pipe(prompt, max_new_tokens=400)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [14]:
output = pipe(prompt, max_new_tokens=400)[0]['generated_text']

if prompt in output:
    print(output.replace(prompt, '').strip())
else:
    print(output.strip())

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [44]:
prompt = "The patient presents with abdominal pain and"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_length=200, do_sample=True, temperature=0.7)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


the patient presents with abdominal pain and...............................................................................................................................................................................................


---------------------------------------------------------

# meditron-7b

In [6]:
model_id = 'I:/meditron-7b'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, pad_token_id=tokenizer.eos_token_id, device=0)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Device set to use cuda:0


In [8]:
notes = ul.get_notes_for_first_n_notes(1)
prompt = ul.sdh_single_prompt(notes[0][3])

In [14]:
output = pipe(prompt, max_new_tokens=400)

if prompt in output:
    print(output[0]['generated_text'].strip().replace(prompt, ''))
else:
    print(output[0]['generated_text'].strip())

Based **only** on the content of the following clinical note, classify the patient’s housing situation.
    
    **Output exactly one of these labels (and nothing else) on its own line:**
    - Stable Housing
    - Difficulty Paying for Housing
    - Without Housing (Undomiciled)
    - Unknown
    
    Clinical note:
    """ 
     
Name:  ___                     Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: MEDICINE
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___
 
Chief Complaint:
Worsening ABD distension and pain 
 
Major Surgical or Invasive Procedure:
Paracentesis

 
History of Present Illness:
___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, 
bioplar, PTSD, presented from OSH ED with worsening abd 
distension over past week.  
Pt reports self-discontinuing lasix and spirnolactone ___ weeks 
ago, because she feels like "they don't do anything" and that 
she "doesn't

---------------------------------------------------------

# meditron3-8b

In [53]:
model_id = 'I:/meditron3-8b'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


---------------------------------------------------------

In [11]:
notes = ul.get_notes_for_first_n_notes(1)
prompt = ul.sdh_prompt(notes[0][3])
# pipe(prompts[0], max_new_tokens=400)

NameError: name 'ul' is not defined

# LlamaCare + MIMIC III

In [3]:
model_id = 'I:/LlamaCare'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(model_id)
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

base_model = "I:/Llama-2-13b-hf"          # o Llama-2-7b-chat-hf
adapter_id = "I:/LlamaCare"             # LoRA de LlamaCare

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16)

tok = AutoTokenizer.from_pretrained(base_model, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(base_model,
                                             quantization_config=bnb_config,
                                             device_map="auto")
model = PeftModel.from_pretrained(model, adapter_id)  # aplica el LoRA

def chat(prompt, max_new_tokens=256):
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    out = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tok.decode(out[0], skip_special_tokens=True)

print(chat("Eres un asistente médico. Explica el manejo inicial de DM2."))


In [ ]:
import random

subject_and_hadm_ids = pd.read_csv('C:/Users/salazarda/Downloads/SDOH_MIMICIII_physio_release.csv')
subject_and_hadm_ids = list(subject_and_hadm_ids.loc[:, ['patient_id', 'note_id']].drop_duplicates().itertuples(index=False, name=None))

notes = ul.get_clinical_notes_mimic3(subject_and_hadm_ids)
notes = random.sample(notes, 5)
# notes = notes[0:50]

sdoh_output = []

for note in notes:
    meta = {
        'subject_id': note[0],
        'hadm_id': note[1],
        'row_id': note[2],
        'charttime': note[3].isoformat() if note[3] else None
    }

    outputs_per_note = meta.copy()  # Start with metadata

    for sdoh in tqdm(['Employment status', 'Housing issues', 'Transportation issues', 'Parental status', 'Relationship status', 'Social support']):
        instruction = {
            'Employment status': 'Employment status: Whether the patient is currently employed, unemployed, underemployed, disability, retired, student, or unknown. LABELS: [employed, unemployed, underemployed, disability, retired, student, unknown]',
            'Housing issues': 'Housing issues: Any mention of financial status, undomiciled, other. LABELS: [financial status, undomiciled, other, unknown]', 
            'Transportation issues': 'Transportation issues: Any reference to transportation difficulties such as distance, resources, other. LABELS: [distance, resources, other, unknown]', 
            'Parental status':'Parental status: Whether the patient has a child under 18 years old. LABELS: [yes, no, unknown]',
            'Relationship status': 'Relationship status: Whether the patient is widowed, divorced, single. LABELS: [married, partnered, widowed, divorced, single, unknown]',
            'Social support': 'Social support: It does include informal or emotional support from family members, friends, or romantic partners unless such support is clearly mediated through a formal care plan by a social worker or case manager. LABELS: [presence, absence, unknown]'
        }

        prompt = ul.sdh_single_prompt(note[4], sdoh, instruction[sdoh])

        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)

        with torch.no_grad():
            generated_ids = model.generate(
                **inputs,
                max_new_tokens=600,
                length_penalty=1.6,
                num_beams=10,
                no_repeat_ngram_size=3,
                temperature=0.8,
                do_sample=True,
                top_k=15,
                top_p=0.95,
                repetition_penalty=2.1,
                early_stopping=True
            )

        output_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        outputs_per_note[sdoh] = output_text

    sdoh_output.append(outputs_per_note)

    

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
import random
time_start = datetime.now()
timestamp = time_start.strftime("%Y%m%d_%H%M%S")

subject_and_hadm_ids = pd.read_csv('C:/Users/salazarda/Downloads/SDOH_MIMICIII_physio_release.csv')
subject_and_hadm_ids = list(subject_and_hadm_ids.loc[:,['patient_id', 'note_id']].drop_duplicates().itertuples(index=False, name=None))
notes = ul.get_clinical_notes_mimic3(subject_and_hadm_ids)
notes = random.sample(notes, 2)
# notes = notes[0:50]

prompts = []
metadata = []

for subject_id, hadm_id, row_id, charttime, note_text in notes:
    prompts.append(ul.sdh_prompt_guevara_v2(note_text))
    metadata.append({
        "subject_id": subject_id,
        "hadm_id": hadm_id,
        "row_id": row_id,
        "charttime": charttime.isoformat() if charttime else None
    })

batch_size = 16
parsed_list = []
final_outputs = []

for i in tqdm(range(0, len(prompts), batch_size)):
    batch_prompts = prompts[i:i+batch_size]
    batch_meta = metadata[i:i+batch_size]
    batch_responses = pipe(batch_prompts, max_new_tokens=400)
    
    for meta, raw, prompt in zip(batch_meta, batch_responses, batch_prompts):
        text = raw[0]['generated_text']
        if prompt in text:
            text = text.replace(prompt, "").strip()
        text = {'text': text}
    
        final_outputs.append({**meta, **text})

ul.save_to_jsonl(final_outputs, model_id, timestamp)

n = len(list(set([i['subject_id'] for i in final_outputs])))

print(f' ... For {n} patients and {len(notes)} notes, it took {datetime.now() - time_start} ... ')


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
notes = ul.get_notes_for_first_n_notes(1)
prompt = ul.sdh_single_prompt(notes[0][3])

In [ ]:
output = pipe(prompt, max_new_tokens=400)

if prompt in output:
    print(output.replace(prompt, '').strip())
else:
    print(output.strip())

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


# Qwen1.5-0.5B

In [1]:
import torch
import utils_llm as ul
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, LlamaForCausalLM

In [3]:
model_id = 'I:/Qwen1.5-0.5B-LoRA-bioinstruct'
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0)

Device set to use cuda:0


In [13]:
notes = ul.get_notes_for_first_n_notes(1)
prompt = ul.sdh_single_prompt(notes[0][3])

In [14]:
prompt

'\n    Based **only** on the content of the following clinical note, classify the patient’s housing situation.\n    \n    **Output exactly one of these labels (and nothing else) on its own line:**\n    - Stable Housing\n    - Difficulty Paying for Housing\n    - Without Housing (Undomiciled)\n    - Unknown\n    \n    Clinical note:\n    """ \n     \nName:  ___                     Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nAttending: ___\n \nChief Complaint:\nWorsening ABD distension and pain \n \nMajor Surgical or Invasive Procedure:\nParacentesis\n\n \nHistory of Present Illness:\n___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, \nbioplar, PTSD, presented from OSH ED with worsening abd \ndistension over past week.  \nPt reports self-discontinuing lasix and spirnolactone ___ weeks \nago, because she feels like "the

In [11]:
output = pipe(prompt, max_new_tokens=400)[0]['generated_text']

if prompt in output:
    print(output.replace(prompt, '').strip())
else:
    print(output.strip())

# Social Status: [Yes/No] - [short evidence sentence]
# Social Support: [Yes/No] - [short evidence sentence]
# Substance Use: [Yes/No] - [short evidence sentence]
# Abdomen: [Yes/No] - [short evidence sentence]
# GU: [Yes/No] - [short evidence sentence]
# Liver: [Yes/No] - [short evidence sentence]
# Pancreas: [Yes/No] - [short evidence sentence]
# Pancreatic Function: [Yes/No] - [short evidence sentence]
# Liver Function: [Yes/No] - [short evidence sentence]
# Liver Function Test: [Yes/No] - [short evidence sentence]
# Liver Function Test 1: [Yes/No] - [short evidence sentence]
# Liver Function Test 2: [Yes/No] - [short evidence sentence]
# Liver Function Test 3: [Yes/No] - [short evidence sentence]
# Liver Function Test 4: [Yes/No] - [short evidence sentence]
# Liver Function Test 5: [Yes/No] - [short evidence sentence]
# Liver Function Test 6: [Yes/No] - [short evidence sentence]
# Liver Function Test 7: [Yes/No] - [short evidence sentence]
# Liver Function Test 8: [Yes/No] - [short

# AWS Bedrock

In [15]:
# !pip install boto3

   ---------------------------------------- 0.0/13.6 MB ? eta -:--:--
   ---------------------------------------  13.4/13.6 MB 64.6 MB/s eta 0:00:01
   ---------------------------------------- 13.6/13.6 MB 57.1 MB/s eta 0:00:00


In [9]:
svc = boto3.client("bedrock", region_name="us-east-1")

# list_foundation_models
resp = svc.list_foundation_models()
print("Foundation models:")
for mdl in resp.get("modelSummaries", []):
    if 'TEXT' in mdl['outputModalities']:
        print(" •", mdl["modelId"])
    else:
        pass
print()

Foundation models:
 • amazon.titan-tg1-large
 • amazon.nova-premier-v1:0:8k
 • amazon.nova-premier-v1:0:20k
 • amazon.nova-premier-v1:0:1000k
 • amazon.nova-premier-v1:0:mm
 • amazon.nova-premier-v1:0
 • amazon.titan-text-premier-v1:0
 • amazon.nova-pro-v1:0:24k
 • amazon.nova-pro-v1:0:300k
 • amazon.nova-pro-v1:0
 • amazon.nova-lite-v1:0:24k
 • amazon.nova-lite-v1:0:300k
 • amazon.nova-lite-v1:0
 • amazon.nova-micro-v1:0:24k
 • amazon.nova-micro-v1:0:128k
 • amazon.nova-micro-v1:0
 • amazon.nova-sonic-v1:0
 • amazon.titan-text-lite-v1:0:4k
 • amazon.titan-text-lite-v1
 • amazon.titan-text-express-v1:0:8k
 • amazon.titan-text-express-v1
 • ai21.jamba-instruct-v1:0
 • ai21.jamba-1-5-large-v1:0
 • ai21.jamba-1-5-mini-v1:0
 • anthropic.claude-instant-v1:2:100k
 • anthropic.claude-instant-v1
 • anthropic.claude-v2:0:18k
 • anthropic.claude-v2:0:100k
 • anthropic.claude-v2:1:18k
 • anthropic.claude-v2:1:200k
 • anthropic.claude-v2:1
 • anthropic.claude-v2
 • anthropic.claude-3-sonnet-202402

In [1]:
import boto3
import json
import utils_llm as ul

In [12]:
def main():
    client = boto3.client("bedrock-runtime", region_name="us-east-1")

    notes = ul.get_notes_for_first_n_notes(1)
    prompt = ul.sdh_prompt(notes[0][3])
    
    # prompt = 'Extract information for: A 62-year-old male school bus driver with a history of hypertension (diagnosed 2015) and hyperlipidemia (diagnosed 2018) presents with a two-day history of intermittent substernal, pressure-like chest pain radiating to the left arm, rated 6/10 in intensity. The discomfort occurs both at rest and with minimal exertion, lasting five to ten minutes each episode, and is accompanied by mild dyspnea on exertion but no diaphoresis or syncope. He denies tobacco, alcohol, or illicit drug use. He lives with his spouse in a two-bedroom apartment and commutes via public transportation. On exam, his blood pressure is 150/90 mmHg, heart rate 88 bpm, respiratory rate 18 breaths/min, SpO₂ 98% on room air, and temperature 36.8 °C. He appears in mild distress but is alert; cardiovascular exam shows a regular rate and rhythm without murmurs or gallops, lungs are clear bilaterally, abdomen is soft and non-tender, and neurologic exam is non-focal. His medications include lisinopril 20 mg daily and atorvastatin 40 mg nightly. The plan is to rule out acute coronary syndrome with ECG and serial troponins, administer aspirin 325 mg PO now and nitroglycerin 0.4 mg SL PRN, continue home medications, monitor vitals and pain every four hours, and consult cardiology for further evaluation.'  

    list_titan_models = ['amazon.titan-tg1-large',
    'amazon.nova-premier-v1:0:8k',
    'amazon.nova-premier-v1:0:20k',
    'amazon.nova-premier-v1:0:1000k',
    'amazon.nova-premier-v1:0:mm',
    'amazon.nova-premier-v1:0',
    'amazon.titan-text-premier-v1:0']
    
    payload = {
        "inputText": prompt,
        "textGenerationConfig": {
            "maxTokenCount": 512,
            "temperature": 0.7
        }
    }
    
    # payload = {
    #     "prompt": "\n\nHuman: " + prompt + "\n\nAssistant:",
    #     'max_tokens_to_sample': 4000
    # }

    # messages = [
    #     {"role": "system",    "content": "You are a clinical NLP assistant."},
    #     {"role": "assistant", "content": "Ready to extract Employment status_, Housing issues, Transportation needs, Parental status, Relationship status, Social support, and Substance Use from a clinical note."},
    #     {"role": "user",      "content": prompt}
    # ]

    # payload = {
    #     "messages": messages,
    #     "temperature": 0.7,
    #     "max_token_count": 256,
    #     "top_p": 0.5
    # }
    
    response = client.invoke_model(
        modelId="amazon.titan-text-premier-v1:0",
        contentType="application/json",
        accept="application/json",
        body=json.dumps(payload).encode("utf-8")
    )
    
    raw = response["body"].read().decode("utf-8")
    try:
        out = json.loads(raw)
        print(json.dumps(out, indent=2, ensure_ascii=False))
    except json.JSONDecodeError:
        print("Raw:", raw)
    
if __name__ == "__main__":
    main()


{
  "inputTextTokenCount": 3470,
  "results": [
    {
      "tokenCount": 130,
      "outputText": "SDH_Employment status_: No - *There is no evidence.*\nSDH_Housing issues: No - *There is no evidence.*\nSDH_Transportation needs: No - *There is no evidence.*\nSDH_Parental status: No - *There is no evidence.*\nSDH_Relationship status: No - *There is no evidence.*\nSDH_Social support: No - *There is no evidence.*\nSDH_Substance Use: Yes - *History of cocaine and heroin use.*",
      "completionReason": "FINISH"
    }
  ]
}


# Flan-T5 XL + MIMIC III

In [2]:
model_id = 'I:/Flan-t5-xl'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
subject_and_hadm_ids = pd.read_csv('C:/Users/salazarda/Downloads/SDOH_MIMICIII_physio_release.csv')
subject_and_hadm_ids = list(subject_and_hadm_ids.loc[:, ['patient_id', 'note_id']].drop_duplicates().itertuples(index=False, name=None))

notes = ul.get_clinical_notes_mimic3(subject_and_hadm_ids)
notes = notes[0:5]

sdoh_output = []

for note in notes:
    meta = {
        'subject_id': note[0],
        'hadm_id': note[1],
        'row_id': note[2],
        'charttime': note[3].isoformat() if note[3] else None
    }

    outputs_per_note = meta.copy()  # Start with metadata

    for sdoh in tqdm(['Employment status', 'Housing issues', 'Transportation issues', 'Parental status', 'Relationship status', 'Social support']):
        instruction = {
            'Employment status': 'Employment status: Whether the patient is currently employed, unemployed, underemployed, disability, retired, student, or unknown. LABELS: [employed, unemployed, underemployed, disability, retired, student, unknown]',
            'Housing issues': 'Housing issues: Any mention of financial status, undomiciled, other. LABELS: [financial status, undomiciled, other, unknown]', 
            'Transportation issues': 'Transportation issues: Any reference to transportation difficulties such as distance, resources, other. LABELS: [distance, resources, other, unknown]', 
            'Parental status':'Parental status: Whether the patient has a child under 18 years old. LABELS: [yes, no, unknown]',
            'Relationship status': 'Relationship status: Whether the patient is widowed, divorced, single. LABELS: [married, partnered, widowed, divorced, single, unknown]',
            'Social support': 'Social support: It does include informal or emotional support from family members, friends, or romantic partners unless such support is clearly mediated through a formal care plan by a social worker or case manager. LABELS: [presence, absence, unknown]'
        }

        prompt = ul.sdh_single_prompt(note[4], sdoh, instruction[sdoh])

        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)

        with torch.no_grad():
            generated_ids = model.generate(
                **inputs,
                max_new_tokens=600,
                length_penalty=1.6,
                num_beams=10,
                no_repeat_ngram_size=3,
                temperature=0.8,
                do_sample=True,
                top_k=15,
                top_p=0.95,
                repetition_penalty=2.1,
                early_stopping=True
            )

        output_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        outputs_per_note[sdoh] = output_text

    sdoh_output.append(outputs_per_note)

    

In [10]:
sdoh_output[0]

{'subject_id': 711,
 'hadm_id': 158767,
 'row_id': 318216,
 'charttime': '2185-04-01T20:34:00',
 'Employment status': '[unknown]',
 'Housing issues': '[unknown]',
 'Transportation issues': '[unknown]',
 'Parental status': 'Respiratory failure',
 'Relationship status': '[unknown]',
 'Social support': '[unknown]'}

# ModernBERT + MIMIC III

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

model_id = "I:/modernBERT-Large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForMaskedLM.from_pretrained(model_id)

text = "The capital of France is [MASK]."
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)

# To get predictions for the mask:
masked_index = inputs["input_ids"][0].tolist().index(tokenizer.mask_token_id)
predicted_token_id = outputs.logits[0, masked_index].argmax(axis=-1)
predicted_token = tokenizer.decode(predicted_token_id)
print("Predicted token:", predicted_token)
# Predicted token:  Paris


# Mistral-7B-Instruct + MIMIC III

In [2]:
model_id = 'I:/Mistral-7B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, pad_token_id=tokenizer.eos_token_id, device=0)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [3]:
import random
time_start = datetime.now()
timestamp = time_start.strftime("%Y%m%d_%H%M%S")

subject_and_hadm_ids = pd.read_csv('C:/Users/salazarda/Downloads/SDOH_MIMICIII_physio_release.csv')
subject_and_hadm_ids = list(subject_and_hadm_ids.loc[:,['patient_id', 'note_id']].drop_duplicates().itertuples(index=False, name=None))
notes = ul.get_clinical_notes_mimic3(subject_and_hadm_ids)
# notes = random.sample(notes, 10)
# notes = notes[0:50]

prompts = []
metadata = []

for subject_id, hadm_id, row_id, charttime, note_text in notes:
    prompts.append(ul.sdh_prompt_guevara_v2(note_text))
    metadata.append({
        "subject_id": subject_id,
        "hadm_id": hadm_id,
        "row_id": row_id,
        "charttime": charttime.isoformat() if charttime else None
    })

batch_size = 16
parsed_list = []
final_outputs = []

for i in tqdm(range(0, len(prompts), batch_size)):
    batch_prompts = prompts[i:i+batch_size]
    batch_meta = metadata[i:i+batch_size]
    batch_responses = pipe(batch_prompts, max_new_tokens=400)
    
    for meta, raw, prompt in zip(batch_meta, batch_responses, batch_prompts):
        text = raw[0]['generated_text']
        if prompt in text:
            text = text.replace(prompt, "").strip()
        text = {'text': text}
    
        final_outputs.append({**meta, **text})

ul.save_to_jsonl(final_outputs, model_id, timestamp)

n = len(list(set([i['subject_id'] for i in final_outputs])))

print(f' ... For {n} patients and {len(notes)} notes, it took {datetime.now() - time_start} ... ')


100%|██████████| 13/13 [23:00<00:00, 106.17s/it]

 ... For 183 patients and 200 notes, it took 0:23:00.314159 ... 


In [5]:
prompts[0]

'\n    Your are a NLP expert assistant. You will be provided with a prompt which consists of three sections: "Input information", "Instructions" and "Clinical note".\n\n    ## Section 1: Input information\n    You will be provided with the following information:\n    1. An arbitrary text sample (CLINICAL_NOTE). The sample is delimited with triple backticks.\n    2. List of categories that we want to search in the text.\n    3. The definition that help you guide to identify the specific label for each category.\n    4. Examples of text samples and their assigned categories. The examples are delimited with triple backticks. These examples are to be used as training data.\n\n    ## Section 2: Instructions (follow the steps in order)\n    1. Analyse the text (CLINICAL_NOTE) and determine whether any of the following six SDOH are explicitly stated based on the definitions labels.  \n    2. For each SDOH, select exactly one label from the bracketed list.  \n       • If the note contains none

In [7]:
final_outputs

[{'subject_id': 711,
  'hadm_id': 158767,
  'row_id': 318216,
  'charttime': '2185-04-01T20:34:00',
  'text': 'Your JSON response:\n    \n      "Employment status": "unknown",\n      "Housing issues": "unknown",\n      "Transportation issues": "unknown",\n      "Parental status": "unknown",\n      "Relationship status": "unknown",\n      "Social support": "unknown"'},
 {'subject_id': 28801,
  'hadm_id': 144188,
  'row_id': 319031,
  'charttime': '2173-02-21T15:16:00',
  'text': 'Your JSON response:\n    \n      "Employment status": "unknown",\n      "Housing issues": "unknown",\n      "Transportation issues": "unknown",\n      "Parental status": "unknown",\n      "Relationship status": "unknown",\n      "Social support": "presence"'},
 {'subject_id': 28207,
  'hadm_id': 117111,
  'row_id': 327606,
  'charttime': '2185-05-26T08:46:00',
  'text': 'Your JSON response:\n    \n      "Employment status": "disability",\n      "Housing issues": "unknown",\n      "Transportation issues": "unkno

In [53]:
import re
records = []

pattern = re.compile(r'"(?:Employment status|Housing issues|Transportation issues|Parental status|Relationship status|Social support)"\s*:\s*"[^"]+"')

for entry in final_outputs:
    base = {"subject_id": entry["subject_id"], "hadm_id": entry["hadm_id"], "charttime": entry["charttime"], "row_id": entry["row_id"]}
    matches = pattern.findall(entry["text"])
    sdhs = {label.split(':')[0].replace('"',''): label.split(':')[1].replace('"','') for label in matches}
    base.update(sdhs)
    records.append(base)
    
df = pd.DataFrame(records)
df = df.map(lambda x: x.lower().strip() if isinstance(x, str) else x)

In [54]:
df

,subject_id,hadm_id,charttime,row_id,Employment status,Housing issues,Transportation issues,Parental status,Relationship status,Social support
0,711,158767.0,2185-04-01t20:34:00,318216,unknown,unknown,unknown,unknown,unknown,unknown
1,28801,144188.0,2173-02-21t15:16:00,319031,unknown,unknown,unknown,unknown,unknown,presence
2,28207,117111.0,2185-05-26t08:46:00,327606,disability,unknown,unknown,yes,married,presence
3,30301,NaN,None,323277,retired,unknown,unknown,no,unknown,presence
4,3100,152037.0,2120-05-16t04:28:00,322894,unknown,unknown,unknown,unknown,unknown,unknown
...,...,...,...,...,...,...,...,...,...,...
195,85941,181409.0,2174-03-22t05:06:00,729125,unknown,unknown,unknown,unknown,unknown,unknown
196,74701,182133.0,2200-04-05t08:19:00,728409,unknown,unknown,unknown,unknown,unknown,unknown
197,50859,144207.0,2130-04-25t10:06:00,737898,unknown,unknown,unknown,unknown,unknown,presence
198,54083,161238.0,2132-11-16t08:54:00,599068,unknown,unknown,unknown,unknown,unknown,unknown


In [55]:
# import re
# records = []

# pattern = re.compile(r'SDH_([^:]+): \[([^\]]+)\]')

# for entry in final_outputs:
#     base = {"subject_id": entry["subject_id"], "hadm_id": entry["hadm_id"], "charttime": entry["charttime"], "row_id": entry["row_id"]}
#     matches = pattern.findall(entry["text"])
#     sdhs = {label.strip(): value for label, value in matches}
#     base.update(sdhs)
#     records.append(base)
    
# df = pd.DataFrame(records)
# df = df.map(lambda x: x.lower() if isinstance(x, str) else x)

In [56]:
sdh_fields = ['Housing issues',
              'Employment status', 
              'Transportation issues', 
              'Parental status',
              'Relationship status', 
              'Social support']
all_map = {}
for field in sdh_fields:
    all_map[field] = {i: i for i in df[f'{field}'].unique()}
all_map

{'Housing issues': {'unknown': 'unknown',
  'financial status': 'financial status',
  'other': 'other',
  'subsidized': 'subsidized'},
 'Employment status': {'unknown': 'unknown',
  'disability': 'disability',
  'retired': 'retired',
  'unemployed': 'unemployed',
  'deferred': 'deferred',
  'employed': 'employed',
  'student': 'student',
  'employed,disabled': 'employed,disabled'},
 'Transportation issues': {'unknown': 'unknown',
  'resources': 'resources',
  'other': 'other'},
 'Parental status': {'unknown': 'unknown', 'yes': 'yes', 'no': 'no'},
 'Relationship status': {'unknown': 'unknown',
  'married': 'married',
  'divorced': 'divorced',
  'partnered': 'partnered',
  'estranged': 'estranged',
  'widowed': 'widowed',
  'family': 'family',
  'brother': 'brother',
  'separated': 'separated',
  'daughter': 'daughter'},
 'Social support': {'unknown': 'unknown',
  'presence': 'presence',
  'absence': 'absence'}}

In [50]:
map_sdoh = {
            # [financial status, undomiciled, other, unknown]
            'Housing issues': {'unknown': 'unknown',
              'financial status': 'financial status',
              'other': 'other',
              'subsidized': 'financial status'},
            # [employed, unemployed, underemployed, disability, retired, student, unknown]
            'Employment status': {'unknown': 'unknown',
              'disability': 'disability',
              'retired': 'retired',
              'unemployed': 'unemployed',
              'deferred': 'unemployed',
              'employed': 'employed',
              'student': 'student',
              'employed,disabled': 'underemployed'},
            # [distance, resources, other, unknown]
            'Transportation issues': {'unknown': 'unknown',
              'resources': 'resources',
              'other': 'other'},
            'Parental status': {'unknown': 'unknown', 'yes': 'yes', 'no': 'no'},
            # [married, partnered, widowed, divorced, single, unknown]
            'Relationship status': {'unknown': 'unknown',
              'married': 'married',
              'divorced': 'divorced',
              'partnered': 'partnered',
              'estranged': 'unknown',
              'widowed': 'widowed',
              'family': 'unknown',
              'brother': 'unknown',
              'separated': 'divorced',
              'daughter': 'unknown'},
            'Social support': {'unknown': 'unknown',
              'presence': 'plus',
              'absence': 'minus'}}

In [51]:
for field in sdh_fields:
    df[f'{field}'] = df[f'{field}'].map(map_sdoh[f'{field}'])
df = df.map(lambda x: x.replace("'", "") if isinstance(x, str) else x)
df_pred = df.rename(columns={'Employment status': 'EMPLOYMENT', 'Housing issues': 'HOUSING', 'Transportation issues': 'TRANSPORTATION', 'Parental status': 'PARENT', 'Relationship status': 'RELATIONSHIP', 'Social support': 'SUPPORT'})
subject_and_hadm_ids = pd.read_csv('C:/Users/salazarda/Downloads/SDOH_MIMICIII_physio_release.csv')
df = subject_and_hadm_ids.iloc[:,5:].copy()

prefixes = set(c.split("_",1)[0] for c in df.columns if "_" in c)

for p in prefixes:
    df[p] = ul.collapse_onehot_group(df, p)
df_real = df.drop(columns=[c for c in df.columns if "_" in c])

df_real_ = pd.concat([subject_and_hadm_ids.iloc[:,0:5], df_real], axis=1)
df_real_ = df_real_.loc[~(df_real_.iloc[:,5:] == 0).all(axis=1)]
df_real_pred = pd.merge(df_real_, df_pred, left_on=['patient_id', 'note_id'], right_on=['subject_id', 'row_id'], how='inner')
df_real_pred.loc[:,['patient_id', 'note_id', 'PARENT_x', 'PARENT_y']].drop_duplicates()

sdh_fields = [
    "EMPLOYMENT",
    "HOUSING",
    "TRANSPORTATION",
    "PARENT",
    "RELATIONSHIP",
    "SUPPORT"
]

for field in sdh_fields:
    df_ = df_real_pred.loc[:,['patient_id', 'note_id', f'{field}_x', f'{field}_y']].drop_duplicates()
    df_ = df_.sort_values(['patient_id', 'note_id', f'{field}_x'],ascending=False).groupby(['patient_id', 'note_id'], as_index=False).first()
    if field == 'PARENT':
        df_ = df_.map(lambda x: 'no' if x == 0 else 'yes')
    else:
        df_ = df_.map(lambda x: 'unknown' if x == 0 else x)
    y_true = df_[f"{field}_x"]
    y_pred = df_[f"{field}_y"]
    # print(f'accuracy for {field}: {accuracy_score(y_true, y_pred)}')
    print(f'{field}... ')
    print(classification_report(y_true, y_pred))
    print('_____________________________________________')

EMPLOYMENT... 
               precision    recall  f1-score   support

   disability       0.75      1.00      0.86         6
     employed       0.56      0.71      0.62         7
      retired       0.56      1.00      0.71         5
      student       0.50      0.50      0.50         4
underemployed       1.00      0.33      0.50         3
   unemployed       0.67      0.44      0.53         9
      unknown       0.97      0.92      0.94        65

     accuracy                           0.84        99
    macro avg       0.71      0.70      0.67        99
 weighted avg       0.86      0.84      0.84        99

_____________________________________________
HOUSING... 
                  precision    recall  f1-score   support

financial status       0.00      0.00      0.00         0
           other       0.00      0.00      0.00         0
            poor       0.00      0.00      0.00         2
         unknown       0.99      0.98      0.98        97

        accuracy           

C:\Users\salazarda\AppData\Local\anaconda3\envs\pytor\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\salazarda\AppData\Local\anaconda3\envs\pytor\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\salazarda\AppData\Local\anaconda3\envs\pytor\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{

---------------------------------------------------------

# OLD CODE

In [79]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

full_prompt = ul.sdh_prompt(notes[0][3])

inputs = tokenizer(full_prompt, return_tensors="pt", truncation=True, max_length=5000).to(device)
output = model.generate(
    **inputs,
    max_new_tokens=5000,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.2,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id 
)

In [80]:
print(tok.decode(output[0], skip_special_tokens=True))


    You are a clinical NLP assistant. Analyze the following clinical note and indicate whether each of the following seven social determinants of health (SDH) is specifically mentioned:
        
    1. **Employment status**: Whether the patient is currently employed, unemployed, retired, on disability, or has a job title or income source.
    2. **Housing issues**: Any mention of homelessness, unstable housing, living in shelters, or housing concerns (e.g., can't afford rent, frequent moves).
    3. **Transportation needs**: Any reference to transportation difficulties, lack of car access, reliance on public transit, missed appointments due to transportation.
    4. **Parental status**: Whether the patient has children or dependents, or is a caregiver to minors.
    5. **Relationship status**: Whether the patient is married, divorced, single, has a partner, or is widowed.
    6. **Social support**: Social support refers to any documented involvement of a social worker or case manager 

In [77]:
extracted_text = tokenizer.decode(output[0], skip_special_tokens=True)

In [78]:
extracted_text

'\n    You are a clinical NLP assistant. Analyze the following clinical note and indicate whether each of the following seven social determinants of health (SDH) is specifically mentioned:\n        \n    1. **Employment status**: Whether the patient is currently employed, unemployed, retired, on disability, or has a job title or income source.\n    2. **Housing issues**: Any mention of homelessness, unstable housing, living in shelters, or housing concerns (e.g., can\'t afford rent, frequent moves).\n    3. **Transportation needs**: Any reference to transportation difficulties, lack of car access, reliance on public transit, missed appointments due to transportation.\n    4. **Parental status**: Whether the patient has children or dependents, or is a caregiver to minors.\n    5. **Relationship status**: Whether the patient is married, divorced, single, has a partner, or is widowed.\n    6. **Social support**: Social support refers to any documented involvement of a social worker or cas

In [49]:
import utils_llm as ul
from tqdm import tqdm
from datetime import datetime

time_start = datetime.now()
n=5
all_outputs = []
notes = ul.get_notes_for_first_n_notes(n)

prompts = []
metadata = []

for subject_id, hadm_id, charttime, note_text in notes:
    prompts.append(ul.sdh_prompt(note_text))
    metadata.append({
        "subject_id": subject_id,
        "hadm_id": hadm_id,
        "charttime": charttime.isoformat() if charttime else None
    })

batch_size = 16
final_outputs = []

for i in tqdm(range(0, len(prompts), batch_size)):
    batch_prompts = prompts[i:i+batch_size]
    batch_meta = metadata[i:i+batch_size]
    
    batch_responses = pipe(batch_prompts, max_new_tokens=400)
    
    for meta, raw in zip(batch_meta, batch_responses):
        parsed = ul.parse_sdh_response(raw[0]['generated_text'])
        final_outputs.append({**meta, **parsed})

ul.save_to_jsonl(final_outputs, model_id)

print(f' ... For {n} patients and {len(notes)} notes, it took {datetime.now() - time_start} ... ')


In [50]:
pipe(prompts[0], max_new_tokens=400)

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


[{'generated_text': '\n    You are a clinical NLP assistant. Analyze the following clinical note and indicate whether each of the following seven social determinants of health (SDH) is specifically mentioned:\n        \n    1. **Employment status**: Whether the patient is currently employed, unemployed, retired, on disability, or has a job title or income source.\n    2. **Housing issues**: Any mention of homelessness, unstable housing, living in shelters, or housing concerns (e.g., can\'t afford rent, frequent moves).\n    3. **Transportation needs**: Any reference to transportation difficulties, lack of car access, reliance on public transit, missed appointments due to transportation.\n    4. **Parental status**: Whether the patient has children or dependents, or is a caregiver to minors.\n    5. **Relationship status**: Whether the patient is married, divorced, single, has a partner, or is widowed.\n    6. **Social support**: Social support refers to any documented involvement of a 

In [9]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained("I:/BioMistral-7B")
print(config.architectures)

['MistralForCausalLM']


In [32]:
from transformers import pipeline

generator = pipeline("text-generation", model="I:/BioMistral-7B")

prompt = "What are the latest treatments for glioblastoma?"
output = generator(
    prompt,
    max_new_tokens=512
)

print(output[0]['generated_text'])


Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What are the latest treatments for glioblastoma?


In [33]:
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0)

Device set to use cuda:0


In [35]:
prompt = "Explain step-by-step how mRNA vaccines work."
pipe(prompts[0], max_new_tokens=400)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': 'Explain how mRNA vaccines work. The mRNA vaccine is a new type of vaccine that is different from the traditional vaccine. The traditional vaccine is made from a dead or weakened virus or bacteria, which can stimulate the body to produce antibodies. The mRNA vaccine is made from a small piece of genetic material (mRNA) that is made in the laboratory. This mRNA is a copy of the genetic material of the virus that causes COVID-19. When the mRNA enters the body, it is taken up by cells, which then produce a protein based on the genetic information contained in the mRNA. This protein is a piece of the “spike” protein found on the surface of the virus that causes COVID-19. The body recognizes this protein as foreign and makes antibodies to it. These antibodies are specifically designed to attach to the spike protein of the virus that causes COVID-19, which helps the body’s immune system to destroy the virus.'}]

In [46]:
prompt = "Explain  how are the latest treatments for glioblastoma."
pipe(prompt, max_new_tokens=400)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': 'Explain step-by-step how are the latest treatments for glioblastoma. The first paragraph is confusing and not clear. The authors should explain the different treatments and then explain how the treatments are improved.'}]

In [45]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch

name = "I:/BioMistral-7B"
tok  = AutoTokenizer.from_pretrained(name)
model = AutoModelForCausalLM.from_pretrained(
    name,
    torch_dtype=torch.float16,
    device_map="auto"
)

msgs = [
    {"role": "system", "content": "You are a helpful biomedical assistant."},
    {"role": "user",   "content": "What are the latest treatments for glioblastoma?"}
]

prompt = tok.apply_chat_template(msgs, add_generation_prompt=True, tokenize=False)

inputs = tok(prompt, return_tensors="pt").to(model.device)
streamer = TextStreamer(tok)          # streams tokens as they appear (optional)

out = model.generate(
    **inputs,
    max_new_tokens=400,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    streamer=streamer                 # real-time printout
)

print(tok.decode(out[0], skip_special_tokens=True))


TemplateError: Conversation roles must alternate user/assistant/user/assistant/...

In [26]:
prompt = ul.sdh_prompt("The patient is a 45-year-old female with no fixed address and a history of alcohol use disorder...")  # Short sample

inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [28]:
pipe(prompt, max_new_tokens=256)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': '\n    You are a clinical NLP assistant. Analyze the following clinical note and indicate whether each of the following seven social determinants of health (SDH) is specifically mentioned:\n        \n    1. **Employment status**: Whether the patient is currently employed, unemployed, retired, on disability, or has a job title or income source.\n    2. **Housing issues**: Any mention of homelessness, unstable housing, living in shelters, or housing concerns (e.g., can\'t afford rent, frequent moves).\n    3. **Transportation needs**: Any reference to transportation difficulties, lack of car access, reliance on public transit, missed appointments due to transportation.\n    4. **Parental status**: Whether the patient has children or dependents, or is a caregiver to minors.\n    5. **Relationship status**: Whether the patient is married, divorced, single, has a partner, or is widowed.\n    6. **Social support**: Social support refers to any documented involvement of a 

In [ ]:
def build_metadata_log(final_outputs):
    yes_count = 0
    no_count = 0
    charttimes = []
    unique_subjects = set()
    
    for record in final_outputs:
        unique_subjects.add(record['subject_id'])
        if record.get('charttime'):
            charttimes.append(record['charttime'])
        
        for var in SDOH_VARIABLES:
            if var in record:
                if record[var]['present'] == "Yes":
                    yes_count += 1
                elif record[var]['present'] == "No":
                    no_count += 1
    
    charttimes_sorted = sorted(charttimes)
    
    log = {
        "processing_time": datetime.now().isoformat(),
        "num_patients": len(unique_subjects),
        "num_notes": len(final_outputs),
        "variables_used": SDOH_VARIABLES,
        "num_yes": yes_count,
        "num_no": no_count,
        "charttime_range": {
            "min": charttimes_sorted[0] if charttimes_sorted else None,
            "max": charttimes_sorted[-1] if charttimes_sorted else None
        }
    }
    return log


In [ ]:
metadata_log = build_metadata_log(final_outputs)

with open("sdoh_processing_log.json", "w", encoding="utf-8") as f:
    json.dump(metadata_log, f, indent=2)

In [16]:
batch_responses

[[{'generated_text': '\n    You are a clinical NLP assistant. Analyze the following clinical note and indicate whether each of the following seven social determinants of health (SDH) is specifically mentioned:\n        \n    1. **Employment status**: Whether the patient is currently employed, unemployed, retired, on disability, or has a job title or income source.\n    2. **Housing issues**: Any mention of homelessness, unstable housing, living in shelters, or housing concerns (e.g., can\'t afford rent, frequent moves).\n    3. **Transportation needs**: Any reference to transportation difficulties, lack of car access, reliance on public transit, missed appointments due to transportation.\n    4. **Parental status**: Whether the patient has children or dependents, or is a caregiver to minors.\n    5. **Relationship status**: Whether the patient is married, divorced, single, has a partner, or is widowed.\n    6. **Social support**: Whether the patient is receiving formal help or assistan

In [10]:
ul.save_to_jsonl(final_outputs)

In [68]:
analyze_sdh_for_subject(subject_id=15005348)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'subject_id': 15005348,
 'CXR': {'present': 'No', 'evidence': 'acute cardiopulmonary process.'},
 'Employment status': {'present': 'No',
  'evidence': 'There is no mention of employment status.'},
 'Housing issues': {'present': 'Yes',
  'evidence': 'The patient is living in a unit with a specified number.'},
 'Transportation needs': {'present': 'No',
  'evidence': 't mentioned - There is no mention of transportation needs.'},
 'Parental status': {'present': 'No',
  'evidence': 't mentioned - There is no mention of parental status.'},
 'Relationship status': {'present': 'No',
  'evidence': 't mentioned - There is no mention of relationship status.'},
 'Social support': {'present': 'Yes',
  'evidence': 'The patient has one brother that she is in touch with.'},
 'Substance Use': {'present': 'Yes',
  'evidence': 'The patient has a history of cocaine and heroin use.'}}

In [3]:
import utils_llm as ul
ul.get_clinical_note(10000032)

' \nName:  ___                     Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nAttending: ___\n \nChief Complaint:\nWorsening ABD distension and pain \n \nMajor Surgical or Invasive Procedure:\nParacentesis\n\n \nHistory of Present Illness:\n___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, \nbioplar, PTSD, presented from OSH ED with worsening abd \ndistension over past week.  \nPt reports self-discontinuing lasix and spirnolactone ___ weeks \nago, because she feels like "they don\'t do anything" and that \nshe "doesn\'t want to put more chemicals in her." She does not \nfollow Na-restricted diets. In the past week, she notes that she \nhas been having worsening abd distension and discomfort. She \ndenies ___ edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, \ndysuria. She had food poisoning a week ago from eatin

In [5]:
def sdh_prompt(note_text):
    return f"""
    You are a clinical NLP assistant. Analyze the following clinical note and indicate whether each of the following seven social determinants of health (SDH) is specifically mentioned:
        
    1. **Employment status**: Whether the patient is currently employed, unemployed, retired, on disability, or has a job title or income source.
    2. **Housing issues**: Any mention of homelessness, unstable housing, living in shelters, or housing concerns (e.g., can't afford rent, frequent moves).
    3. **Transportation needs**: Any reference to transportation difficulties, lack of car access, reliance on public transit, missed appointments due to transportation.
    4. **Parental status**: Whether the patient has children or dependents, or is a caregiver to minors.
    5. **Relationship status**: Whether the patient is married, divorced, single, has a partner, or is widowed.
    6. **Social support**: Whether the patient is receiving formal help or assistance from a **social worker**, also extracts the name of the service.
    7. **Substance Use**: Any mention of alcohol, drug, or tobacco use, including current use, past use, or explicit denial of use.
    
    Answer with **"Yes" or "No"** for each item, and include a **short evidence sentence**. If not mentioned, say: *There is no evidence.* 
    
    ---
    
    Now analyze the following clinical note:
    
    \"\"\"
    {note_text}
    \"\"\"
    
    Respond in this format:
    
    Employment status: [Yes/No] - [short evidence sentence]
    Housing issues: [Yes/No] - [short evidence sentence]
    Transportation needs: [Yes/No] - [short evidence sentence]
    Parental status: [Yes/No] - [short evidence sentence]
    Relationship status: [Yes/No] - [short evidence sentence]
    Social support: [Yes/No] - [short evidence sentence]
    Substance Use: [Yes/No] - [short evidence sentence]
    """

In [4]:
prompt = ul.sdh_prompt(ul.get_clinical_note(10000032))

In [5]:
prompt

'\n    You are a clinical NLP assistant. Analyze the following clinical note and indicate whether each of the following seven social determinants of health (SDH) is specifically mentioned:\n        \n    1. **Employment status**: Whether the patient is currently employed, unemployed, retired, on disability, or has a job title or income source.\n    2. **Housing issues**: Any mention of homelessness, unstable housing, living in shelters, or housing concerns (e.g., can\'t afford rent, frequent moves).\n    3. **Transportation needs**: Any reference to transportation difficulties, lack of car access, reliance on public transit, missed appointments due to transportation.\n    4. **Parental status**: Whether the patient has children or dependents, or is a caregiver to minors.\n    5. **Relationship status**: Whether the patient is married, divorced, single, has a partner, or is widowed.\n    6. **Social support**: Whether the patient is receiving formal help or assistance from a **social wo

In [9]:
pipe(prompt, max_new_tokens=400)[0].keys()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


dict_keys(['generated_text'])